In [3]:
import glob, os
import subprocess
import pandas as pd


In [4]:
def make_dict(sub):
    qa_dict[sub] = {
        "TASKCHECK": None,
        "TASKCOUNT": 0,
        "ERRORLOGS" : [],
    }

In [2]:
qa_dict = {}
SUBJECTS = glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/bbx/derivatives", "sub-*"))
cmdA = "grep 'ERROR' /projects/niblab/bids_projects/Experiments/bbx/derivatives/sub-*/ses-1/func/Analysis/feat1/task*/report_log.html"
#errors = os.system(cmdA)
ERRORS = subprocess.check_output(cmdA, shell=True)
ERRORS = ERRORS.decode("utf-8")
for sub_path in sorted(SUBJECTS):
    subID = sub_path.split("/")[-1]
    if subID not in qa_dict:
        make_dict(subID)
    print(">>>>------> ", subID)
    task_count = 0
    TASKFOLDERS = glob.glob(os.path.join(sub_path, "ses-1/func/Analysis/feat1/task*"))
    #print(TASKFOLDERS)
    if not TASKFOLDERS:
        TASK_CHECK = "***FAIL"
    else:
        for task_dir in TASKFOLDERS:
            #print(task_dir)
            task_count = task_count + 1
        if task_count == 4:
            TASK_CHECK = "PASS"
        else:
            TASK_CHECK = "*PARTIAL FAIL"
    ##check for zstats 
    qa_dict[subID]["TASKCHECK"] = TASK_CHECK
    qa_dict[subID]["TASKCOUNT"] = task_count 

        

In [ ]:
for err in ERRORS.split("\n"):
    for word in err.split("/"):
        if "sub-" in word:
            subID = word
    for word in err.split("/"):
        if ".feat" in word:
            run_dir = word
    if run_dir not in qa_dict[subID]["ERRORLOGS"]:
        qa_dict[subID]["ERRORLOGS"].append(run_dir)
    else:
        pass

In [5]:
pd.DataFrame.from_dict(qa_dict, orient='index')

NameError: name 'qa_dict' is not defined

In [ ]:
qa_path = os.path.join("/projects/niblab/bids_projects/Experiments/bbx/derivatives/quality_ana", "feat1_qa")

In [ ]:
os.makedirs(qa_path)

In [ ]:
with open(qa_path+"/ses-1_errors.txt", 'a') as f:
    for sub in qa_dict:
        for err_ in qa_dict[sub]["ERRORLOGS"]:
             if not err_:
                pass
             else:
                txt1= "ERRORS FOUND: \nSUBJECT: %s\n"%(sub)
                txt2 = "DIRECTORY: %s \n"%(err_) 
                f.write(txt1)
                f.write(txt2)
        if qa_dict[sub]["TASKCHECK"] != "PASS":
            txt3 = "SUBJECT %s FAILED THE TASK DIRECTORY COUNT\n"%(sub)
            txt4 = "THE PROGRAM COUNTED %s TASK DIRECTORIES MADE\n"%(qa_dict[sub]["TASKCOUNT"])
            print(txt3)
            print(txt4)
            f.write(txt3)
            f.write(txt4)

QUALITY ANALYSIS NOTES:  
  
-- sub-018 is missing files  
-- sub-002 is missing run-04 onsets  
-- sub-004 is missing run-03 onsets
-- sub-009 is missing run-03/04 onsets
